In [1]:
# Import external libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Set number of bits per cell
bpc = 2

In [3]:
# Analyze reliability, bandwidth, energy
data = []
for att in range(10, 250, 20):
    # Load and compute data
    postread = np.loadtxt(f"data/postread_{bpc}bpc_{att}.csv", delimiter=',', dtype=np.int8)
    postread_cer = np.not_equal(postread, [[i % (2**bpc) for i in range(16)]]*65536).mean()
    dt = np.loadtxt(f"data/dt_{bpc}bpc_{att}.csv", delimiter=',')
    ivdd = np.loadtxt(f"data/vdd_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivdd_dac = np.loadtxt(f"data/vdd_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivsa = np.loadtxt(f"data/vsa_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivddio = np.loadtxt(f"data/vddio_power_{bpc}bpc_{att}.csv", delimiter=',')
    ivddio_dac = np.loadtxt(f"data/vddio_power_{bpc}bpc_{att}.csv", delimiter=',')
    power_tot = ivdd * 0.9 + ivdd_dac * 0.9 + ivsa * 0.9 + ivddio_dac * 3.3 + ivddio * 3.3
    energy_tot = power_tot * dt
    bw = 2**16 * 48 * bpc / dt
    energy_per_bit = energy_tot / 2**16 / 48 / bpc


    # Load data into confmat
    confmat = np.zeros(shape=(2**bpc, 2**bpc))
    for i in range(65536):
        for j in range(16):
            confmat[j % (2**bpc)][postread[i][j]] += 1
    confmat = confmat.astype(int)
    print(confmat)

    # Symbol error magnitudes
    mae = sum(abs(i - j) * confmat[i][j] for i in range(2**bpc) for j in range(2**bpc)) / confmat.sum()
    rmse = np.sqrt(sum((i - j)**2 * confmat[i][j] for i in range(2**bpc) for j in range(2**bpc)) / confmat.sum())

    # Function to get Gray code string
    getgray = lambda x : format(2**(2**bpc) + x^x//2, f"0{2**bpc-1}b")[1:]

    # Function to get Gray code error rate for each possible symbol confusion
    grayerr = np.array([[sum([int(ic != jc) for ic, jc in zip(getgray(i), getgray(j))]) for i in range(2**bpc)] for j in range(2**bpc)]) / bpc

    # BER
    ber = (grayerr * confmat).sum() / confmat.sum()
    
    # Compose dataframe
    data.append({"att": att, "bpc": bpc,  "dt": dt, "power_tot": power_tot, "energy_per_bit": energy_per_bit, "bw": bw, "cer": postread_cer, "ber": ber, "mae": mae, "rmse": rmse})
data = pd.DataFrame(data)
data

[[252515   1376   1247   7006]
 [ 20368 239569   1204   1003]
 [ 50701    772 208123   2548]
 [     6      9    156 261973]]
[[253978    768    653   6745]
 [  2445 256024    629   3046]
 [ 32157    581 224188   5218]
 [     6      9     32 262097]]
[[254655    713    410   6366]
 [  1071 257223    569   3281]
 [ 21026    575 234849   5694]
 [     6      9     21 262108]]
[[254896    763    316   6169]
 [   672 257628    622   3222]
 [ 14540    539 241396   5669]
 [     6      9     31 262098]]
[[254964    838    255   6087]
 [   623 257473    625   3423]
 [ 15699    475 241149   4821]
 [     7      8     26 262103]]
[[254958    854    243   6089]
 [   538 257452    604   3550]
 [ 13294    479 243606   4765]
 [     7      9     22 262106]]
[[254882    967    245   6050]
 [   530 257256    604   3754]
 [  8706    437 248294   4707]
 [     7      9     18 262110]]
[[254871    976    279   6018]
 [   514 257083    611   3936]
 [  9588    464 247870   4222]
 [     7      8     28 262101]]


,att,bpc,dt,power_tot,energy_per_bit,bw,cer,ber,mae,rmse
0,10,2,0.31820058822631836,0.025916,1.310730e-09,1.977198e+07,0.082394,0.066450,0.146275,0.536107
1,30,2,0.49591565132141113,0.026325,2.075030e-09,1.268654e+07,0.049867,0.042035,0.096947,0.451645
2,50,2,0.6294693946838379,0.026718,2.673170e-09,9.994856e+06,0.037900,0.030740,0.073634,0.396555
3,70,2,0.7402698993682861,0.025597,3.011866e-09,8.498868e+06,0.031050,0.024149,0.060077,0.360428
4,90,2,0.8342056274414062,0.025359,3.362493e-09,7.541853e+06,0.031363,0.024925,0.061474,0.365127
5,110,2,0.916248083114624,0.025480,3.710816e-09,6.866542e+06,0.029043,0.022674,0.056974,0.352795
6,130,2,0.9977076053619385,0.025758,4.084728e-09,6.305912e+06,0.024828,0.018476,0.048506,0.327741
7,150,2,1.0796642303466797,0.025918,4.447703e-09,5.827234e+06,0.025416,0.019294,0.050079,0.333012
8,170,2,1.1539311408996582,0.025877,4.746205e-09,5.452194e+06,0.023887,0.017856,0.047201,0.324530
9,190,2,1.2266960144042969,0.026603,5.186926e-09,5.128782e+06,0.022738,0.016624,0.044862,0.317371
